In [1]:
!pip install geopandas openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.5 MB/s eta 0:00:00
  Using cached pyproj-3.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 kB 11.5 MB/s eta 0:00:00
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 63.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 51.8 MB/s eta 0:00:00:00:0100:01
Using cached pyproj-3.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import geopandas as gpds
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
from shapely import wkt
from openpyxl import load_workbook
import geopandas as gpds
import sys
import re
import os

In [4]:
# FieldMap visualizer based on two Excel files - one with the plot-based data, 
# and another that has the geometric representation of the field plots
# The user first reads-in the data file- and selects a location
# based on the selected location, the data file is filtered for that location
# followed by the Field map file being opened and used for extracting the polygons for each plot for the selected trial

# Define the directory path
directory = "/projects/illinois/aces/cropsci/ese/SoyData/advaymb2/"

# Search for files matching '2023' or '2024' in the directory
file_pattern = re.compile(r"202[3-4]*_Advancement.xlsx")
matching_files = [f for f in os.listdir(directory) if file_pattern.match(f)]

# Ensure there are matching files
if not matching_files:
    raise FileNotFoundError("No matching files found for 2023 or 2024.")

# Display available options
print("Available files:")
for i, file in enumerate(matching_files, 1):
    print(f"{i}: {file}")

# Ask the user to select a file by choosing 3 or 4
while True:
    user_choice = input("Enter 3 for 2023 or 4 for 2024: ").strip()
    
    if user_choice in ["3", "4"]:
        selected_year = f"202{user_choice}"
        selected_file = next((f for f in matching_files if f.startswith(selected_year)), None)
        if selected_file:
            break
        else:
            print(f"No file found for {selected_year}, please try again.")
    else:
        print("Invalid input. Please enter 3 or 4.")

# Construct full file path
file_path = os.path.join(directory, selected_file)

print(f"Selected file: {file_path}")
#advaymb2-projects/SoyData/advaymb2/output/Copy of 2023 Perry Soybean map.xlsx

Available files:
1: 2024_Advancement.xlsx
2: 2023_Advancement.xlsx


Enter 3 for 2023 or 4 for 2024:  4


Selected file: /projects/illinois/aces/cropsci/ese/SoyData/advaymb2/2024_Advancement.xlsx


In [14]:
# Characters that represent missing data
na_characters = [-9, "-9", ""]

# Load the Excel file
xls = pd.ExcelFile(file_path)

# List available sheets
sheet_names = xls.sheet_names
print("Available sheets in the Excel file:")
for i, sheet in enumerate(sheet_names, 1):
    print(f"{i}: {sheet}")

# Prompt user to select a sheet by index
while True:
    sheet_index = input("\nEnter the index number of the sheet you want to load: ").strip()
    
    if sheet_index.isdigit():
        sheet_index = int(sheet_index)
        if 1 <= sheet_index <= len(sheet_names):
            selected_sheet = sheet_names[sheet_index - 1]
            print(f"\nYou selected: {selected_sheet}")
            break
        else:
            print(f"Please enter a number between 1 and {len(sheet_names)}.")
    else:
        print("Invalid input. Please enter a valid number.")

# Read the selected sheet
df = pd.read_excel(xls, sheet_name=selected_sheet, header=0, na_values=na_characters, skiprows=0)


Available sheets in the Excel file:
1: list
2: MO
3: NE
4: IA
5: VT
6: UM
7: list (2)
8: agro



Enter the index number of the sheet you want to load:  5



You selected: VT


In [15]:
df.columns

Index([    'QR code',         'ent',        'name',      'FPhilm',
             'Trial',        'test',        'plot',         'FP2',
                'Ht',         'Pod',         'Mat',         'Lod',
               'Yld',        'oil ',     'protein',       'notes',
               'rep',         'loc',        'locc',         'rng',
               'row',         'run',       'field',     'Sds/env',
       'Unnamed: 24',      'source', 'Unnamed: 26', 'Unnamed: 27',
           'parents',       'trait',             0,       '2.5.7',
              'geno',       '1,2,3',   'Reg tests',     'Variety',
       'Unnamed: 36'],
      dtype='object')

In [16]:
### Ensure the column 'loc' exists
# if it does capture the names of the available locations
if 'loc' in df.columns:
    locations_available = df['loc'].dropna().unique()  # Drop NaNs before getting unique values
    print("Unique locations:", locations_available)
else:
    raise KeyError("Column 'loc' not found in the dataframe.")

# Display available options for locations
print("Available locations:")
for i, loc in enumerate(locations_available, 1):
    print(f"{i}: {loc}")
    
# Ask the user to select a location by choosing an index
while True:
    user_choice = input("\nEnter the index number of the location you want: ").strip()
    
    # Ensure user input is a valid integer
    if user_choice.isdigit():
        user_choice = int(user_choice)
        
        # Check if the index is within the valid range
        if 1 <= user_choice <= len(locations_available):
            selected_location = locations_available[user_choice - 1]  # Convert index to value
            break
        else:
            print(f"Invalid choice. Please enter a number between 1 and {len(locations_available)}.")
    else:
        print("Invalid input. Please enter a valid number.")

# Print selected location
print(f"\nYou selected: {selected_location}")
filtered_data = df[df['loc'] == selected_location]

Unique locations: ['Belleville' 'Dekalb' 'Freeport' 'Goodfield' 'Monmouth' 'Perry'
 'St. Peter']
Available locations:
1: Belleville
2: Dekalb
3: Freeport
4: Goodfield
5: Monmouth
6: Perry
7: St. Peter



Enter the index number of the location you want:  1



You selected: Belleville


In [17]:
# Find the Map file corresponding to the data file
input_folder = "/projects/illinois/aces/cropsci/ese/SoyData/advaymb2/input/" 
pattern = re.compile(rf"{selected_year} {selected_location} Soybean [Mm]ap\.xlsx")

# Search for the map file in the directory
matching_files = [f for f in os.listdir(input_folder) if pattern.fullmatch(f)]

if not matching_files:
    raise FileNotFoundError(f"No map file found for {selected_year} {selected_location}")

# Assume first match is the desired one
map_file_name = matching_files[0]
map_file_path = os.path.join(input_folder, map_file_name)

# Load the map file
selected_loc_map = pd.read_excel(map_file_path, sheet_name=0)  # Adjust sheet_name if needed
print(f"Loaded map file: {map_file_path}")

Loaded map file: /projects/illinois/aces/cropsci/ese/SoyData/advaymb2/input/2024 Belleville Soybean map.xlsx


In [26]:
dataDictionary = {}
dataDictionary = filtered_data.columns
dataDictionary = {'QR code':'QR code for the plot',
            'ent':'entry id for the trial and location',
            'name': 'line identifier' ,      
            'FPhilm':'Flower color(P:purple,W:White,M:Mix, 1 character)-Pubescence(Lt,Gi,2characters)-Hilum color: 6genes(I,T,R,O,W1,Wp),2 to 4 characters',
            'trial': 'year(2digits)+trial type(2digits)+ some string( 1 letter)',
            'test': 'trial identifier + 2 or 3 letters for Location',        
            'plot': 'plot identifier unique to each location',         
            'FP2': 'Flower color, often empty in the files',
            'Ht': 'Height, measured',         
            'Pod': 'Pod Color: b,m,t',         
            'Mat': 'estimated maturity date: 9- is for September, the two digits that follow are the calendar days of September, if bleeds over 30, implies October',         
            'Lod': 'lodging score, lower is better',
            'Yld': 'Yield, standardized to 13% moisture by combine',         
            'oil': 'combine measured - NIR based',     
            'protein': 'Combine measured- NIR based',       
            'notes': 'free text area for note taker',
            'rep': 'replicate within location, often 2 for VT trials',         
            'loc': 'location name',        
            'locc': 'location code, 1 to 3 capital letters',         
            'rng': 'range where the plot is located',
            'row': 'row where the plot is located',         
            'run': 'not sure what this is',       
            'field': 'field identifier within each location',     
            'Sds/env': 'Seeds planted- aimed at 800 at later stage trials, can be as low as 200 for early trials',
            'source': 'line identifier from previous season seed increase', 
            'Unnamed: 25':'not sure - drop', 
            'Unnamed: 26': 'not sure - drop',     
            'parents': 'parents of the tested line- female x male',
            'trait': 'Breeding objective/selection target: HOLL :High-Oleic-Low-Linoleic, Rhg1, Rhg4, etc. ',             
            '0': 'SCN isolate with Type-0 cyst count on the variety based on GH inoculation',       
            '2.5.7':'SCN isolate with Type-2.5.7 cyst count on the variety based on GH inoculation',        
            'geno':'Genotypes of the SCN resistance alleles.There are 4 total genes, 2 derived from PI88788, and another 2 derived from Peking, listed as 4 capital letters, each letter chooses from S(usceptible)/R(esistant)',
            '1,2,3': 'not sure',   
            'Reg tests' : 'Name of the trial type: UT- uniform trials, Sp,PT, MSmC, etc.',     
            'Variety' : 'Commercial Names of the released varieties', 
            'Unnamed: 35' : 'Randomization index'}

In [18]:
# functions for I/O

# Find the starting row index that contains plot identifiers 1, 2, and 3
def find_head(dataframe):
    for index, row in dataframe.iterrows():
        row_values = set(row.dropna().astype(str))
        if {'1.0', '2.0', '3.0'}.issubset(row_values):
            return index + 1
    raise ValueError("Could not find a row containing '1', '2', and '3'")

# Find the starting column index (assumes columns are named like 'Range_5', 'Col_6', etc.)
def find_col(dataframe):
    for col_name, col in dataframe.items():
        col_values = set(col.dropna().astype(str))
        if {'1.0', '2.0', '3.0'}.issubset(col_values):
            match = re.search(r'(\d+)$', col_name)  # Extract trailing number
            if match:
                return int(match.group(1))
    raise ValueError("Could not find a column containing '1', '2', and '3'")

# Create WKT polygon from string coordinates (not recommended for numeric math)
def create_polygon(col, row):
    col = int(col)
    row = int(row)
    return (
        f"POLYGON (({col} {row}, {col+1} {row}, {col+1} {row+1}, "
        f"{col} {row+1}, {col} {row}))"
    )

In [19]:
row = str(find_head(selected_loc_map))
col = str(find_col(selected_loc_map))
create_polygon(col,row)

# will use file_1 to get the row & col coordinates for location
# will use file_2 get the geometry of the trial in location
# generate a polygon file per location
# wisua


ValueError: Could not find a row containing '1', '2', and '3'

In [59]:
polygon_records = []

def write_polygon_records(value, file_path=selected_location, sheet_name=selected_sheet, col=col, row=row):
    # Load workbook
    wb = load_workbook(file_path)
    
    # Get the correct worksheet
    ws = wb[sheet_name] if sheet_name in wb.sheetnames else wb.active
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=19, max_col=19):
        for cell in row:
          print()
    # Record this update for CSV export
    polygon_records.append({
        "sheet": sheet_name,
        "row": row,
        "col": col,
        "polygon_wkt": value
    })

    # Save the Excel workbook back if needed (optional)
    # wb.save(file_path)  # You can skip this if not updating the .xlsx

    # ---- CSV export logic below ----

    # Build CSV file path based on original file name
    base_name = os.path.splitext(os.path.basename(file_path))[0]  # no extension
    output_folder = os.path.dirname(file_path)
    output_file_name = base_name + "_polygons.csv"
    output_csv_path = os.path.join(output_folder, output_file_name)

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(polygon_records)
    df.to_csv(output_csv_path, index=False)

    print(f"✅ Polygon saved to cell ({row}, {col}) in sheet '{sheet_name}'")
    print(f"📁 Polygon record written to: {output_csv_path}")



In [60]:
def fill_cell(col, row, value):
  wb = selected_loc_map
  ws = wb["A"]
  for row in ws.iter_rows(min_row=find_row, max_row=ws.max_row, min_col=find_col, max_col=19):
    for cell in row:
      print()
  ws.cell(row, col).value = value
 # wb.save("/projects/illinois/aces/cropsci/ese/SoyData/advaymb2/output/Copy of" + selected_location + selected_year +".xlsx")

input_path = map_file_path  # The path you used to load the map
base_name = os.path.splitext(os.path.basename(input_path))[0]  # e.g. "2023 Goodfield Soybean Map"
folder = os.path.dirname(input_path)  # Get the input folder path

# Construct the output file name
output_file_name = base_name + "_polygons.csv"
output_path = os.path.join(folder, output_file_name)

# Save the DataFrame or GeoDataFrame to CSV
your_dataframe.to_csv(output_path, index=False)

print(f"Polygon data written to: {output_path}")

NameError: name 'your_dataframe' is not defined

In [35]:
fill_cell(col,row,1)

KeyError: 'A'

In [20]:
def mapper(field_name, write_to):
  output_name = selected_year + selected_location + " MapFile-polygons.txt"
  field_wb = selected_loc_map
  field_ws = field_wb["A"]
  wb = field_wb
  ws = wb[""]

  beginning_row = find_head(file_name)
  beginning_col = find_col(file_name)
  idx = 1
  for row in ws.iter_rows(max_row = ws.max_row - 1, min_col=19, max_col=19):
    for cell in row:
      if cell.value == field_name:
        #print(str(ws.cell(idx, 7).value))
        starting_row = beginning_row
        while(starting_row < 60):
          starting_row += 1
          col = beginning_col
          while(col < 30):
            col += 1
            #print("plot: " + str(field_ws.cell(starting_row, col).value))
            #print(str(ws.cell(idx, 7).value))
            if str(field_ws.cell(starting_row, col).value) == str(int(ws.cell(idx, 7).value)):
              #print("plot: " + str(int(ws.cell(idx, 7).value)))
              #print("row: " + str(starting_row))
              #print("col: " + str(col))
              ws.cell(idx, 8).value = ("POLYGON (("+str(field_ws.cell(beginning_row, col).value)+" "
                                       +str(field_ws.cell(starting_row, beginning_col).value)+", "
                                       + str(field_ws.cell(beginning_row, col).value + 1) +" "
                                       +str(field_ws.cell(starting_row, beginning_col).value)+", "
                                       + str(field_ws.cell(beginning_row, col).value + 1) +" "
                                       + str(field_ws.cell(starting_row, beginning_col).value + 1) +", "
                                       +str(field_ws.cell(beginning_row, col).value)+" "
                                       + str(field_ws.cell(starting_row, beginning_col).value + 1) +", "
                                       +str(field_ws.cell(beginning_row, col).value)+" "
                                       +str(field_ws.cell(starting_row, beginning_col).value)+"))")


    idx += 1
  wb.save("/projects/illinois/aces/cropsci/ese/SoyData/advaymb2/output/"+ output_name)


  #print(temp_data.at[starting_row, starting_col])

#mapper("Belleville", "blah")

#mapper("St. Peter", "")

In [23]:
import os
import openpyxl
output_name = f"{selected_year}_{selected_location}_MapFile_polygons.csv"

field_wb = selected_loc_map
field_ws = field_wb["A"]
wb = field_wb
ws = wb[""]

def mapper(field_name,output_name):    

    output_name=output_name

    beginning_row = find_head(file_name)  # Ensure this function exists
    beginning_col = find_col(file_name)  # Ensure this function exists
    idx = 1

    for row in ws.iter_rows(max_row=ws.max_row - 1, min_col=19, max_col=19):
        for cell in row:
            if cell.value == field_name:
                starting_row = beginning_row
                while starting_row < 60:
                    starting_row += 1
                    col = beginning_col
                    while col < 30:
                        col += 1
                        plot_value = field_ws.cell(starting_row, col).value
                        ws_value = ws.cell(idx, 7).value

                        if ws_value is not None:
                            ws_value = int(float(ws_value))  # Safe conversion

                        if str(plot_value) == str(ws_value):
                            # Create the polygon string
                            ws.cell(idx, 8).value = (
                                f"POLYGON (({field_ws.cell(beginning_row, col).value} "
                                f"{field_ws.cell(starting_row, beginning_col).value}, "
                                f"{field_ws.cell(beginning_row, col).value + 1} "
                                f"{field_ws.cell(starting_row, beginning_col).value}, "
                                f"{field_ws.cell(beginning_row, col).value + 1} "
                                f"{field_ws.cell(starting_row, beginning_col).value + 1}, "
                                f"{field_ws.cell(beginning_row, col).value} "
                                f"{field_ws.cell(starting_row, beginning_col).value + 1}, "
                                f"{field_ws.cell(beginning_row, col).value} "
                                f"{field_ws.cell(starting_row, beginning_col).value}))"
                            )
        idx += 1

    # Save the workbook to the output directory
    output_path = os.path.join("/u/ese/projects-ese/SoyData/advaymb2/output/", output_name)
    wb.save(output_path)

    print(f"File saved: {output_path}")

# Example call (Make sure `find_head` and `find_col` exist)
# mapper("Belleville", "blah", "2023", "Goodfield", selected_loc_map, "input.xlsx")


KeyError: 'A'

In [22]:
mapper(selected_location, "")

KeyError: 'A'